In [1]:
# The working finite field
r = 2^64 - 2^32 + 1
Fr = GF(r)

# The polynomial Ring over Fr
R.<x> = PolynomialRing(Fr)

# Generates a random polynomial of degree less than n
def rand_poly(n):
    return R([R.base_ring().random_element() for _ in range(n)])

# Computes the polynomial interpolating points
def lagrange_interpolation(points):
    x = R.gen()
    f = R(0)
    for i in range(len(points)):
        xi, yi = Fr(points[i][0]), Fr(points[i][1])
        li = R(1)
        for j in range(len(points)):
            if i != j:
                xj = Fr(points[j][0])
                li *= (x - xj) / (xi - xj)
        f += yi * li
    return f

# Computes the degree of the polynomial interpolating the evaluation vector f_eval over the domain {w^i}_i=0,..,N-1
def deg(f_eval, w, N):
    f_eval_size = len(f_eval)
    assert N % f_eval_size == 0
    domain = [w^(i*N//f_eval_size) for i in range(f_eval_size)]
    f = lagrange_interpolation([(x,y) for x,y in zip(domain, f_eval)])
    return f.degree()
    
# Checks if the points P1, P2, P3 lie on the same line
def are_collinear(P1, P2, P3):
    x1, y1 = P1
    x2, y2 = P2
    x3, y3 = P3    
    return (x2 - x1) * (y3 - y1) == (y2 - y1) * (x3 - x1)

In [2]:
# Merkle tree implementation adapted from https://github.com/aszepieniec/stark-anatomy/blob/master/code/merkle.py
from hashlib import sha256

class Merkle:
    H = sha256

    def commit_( leafs ):
        assert(len(leafs) & (len(leafs)-1) == 0), "length must be power of two"
        if len(leafs) == 1:
            return leafs[0]
        else:
            return Merkle.H(Merkle.commit_(leafs[:len(leafs)//2]) + Merkle.commit_(leafs[len(leafs)//2:])).digest()
    
    def open_( index, leafs ):
        assert(len(leafs) & (len(leafs)-1) == 0), "length must be power of two"
        assert(0 <= index and index < len(leafs)), "cannot open invalid index"
        if len(leafs) == 2:
            return [leafs[1 - index].hex()]
        elif index < (len(leafs)/2):
            return Merkle.open_(index, leafs[:len(leafs)//2]) + [Merkle.commit_(leafs[len(leafs)//2:]).hex()]
        else:
            return Merkle.open_(index - len(leafs)//2, leafs[len(leafs)//2:]) + [Merkle.commit_(leafs[:len(leafs)//2]).hex()]
    
    def verify_( root, index, path, leaf ):
        assert(0 <= index and index < (1 << len(path))), "cannot verify invalid index"
        if len(path) == 1:
            if index == 0:
                return root == Merkle.H(leaf + bytes.fromhex(path[0])).digest()
            else:
                return root == Merkle.H(bytes.fromhex(path[0]) + leaf).digest()
        else:
            if index % 2 == 0:
                return Merkle.verify_(root, index >> 1, path[1:], Merkle.H(leaf + bytes.fromhex(path[0])).digest())
            else:
                return Merkle.verify_(root, index >> 1, path[1:], Merkle.H(bytes.fromhex(path[0]) + leaf).digest())
                
    def commit( data_array ):
        return Merkle.commit_([Merkle.H(hex(da).encode()).digest() for da in data_array]).hex()

    def open( index, data_array ):
        return Merkle.open_(index, [Merkle.H(hex(da).encode()).digest() for da in data_array])

    def verify( root, index, path, data_element ):
        return Merkle.verify_(bytes.fromhex(root), index, path, Merkle.H(bytes(hex(data_element).encode())).digest())

# We check correctness of the Merkle Tree implementation
leafs = [Fr(i) for i in range(1 << 10)]
index = randint(0, len(leafs)-1)
root = Merkle.commit( leafs )
path = Merkle.open(index, leafs)
assert Merkle.verify(root, index, path, leafs[index])

In [3]:
# Computes a primitive N-th root of unity in Fr 
def setup(Fr, N):
    r = Fr.characteristic()
    (_,m) = factor(r - 1)[0]
    assert (r-1) % 2^m == 0 
    assert 2^m % N == 0 
    gamma = Fr.primitive_element()^((r-1)/2^m)
    assert gamma.multiplicative_order() == 2^m
    w = gamma^(2^m/N)
    return w

# Splits and Folds a polynomial f using the provided challenge
def split_and_fold(f, challenge):
    f_even = R(f.coefficients()[::2])
    f_odd  = R(f.coefficients()[1::2])
    # We check that split is correct
    assert f == f_even(x^2) + x*f_odd(x^2)
    f_fold = R(f_even + challenge*f_odd)
    return f_fold
    
# Commits to a polynomial f interactively
def commit_ip(f, w, N, rounds):
    
    f_commits = []
    f_fold_commits = []
    f_evals = []
    f_fold_evals = []
    fold_challenges = []
    
    for round in range(rounds):
    
        # The Prover computes and sends each f_commit to the Verifier
        f_eval = [f(w^i) for i in range(N)]
        f_commit = Merkle.commit( f_eval )
        
        # The Verifier picks a random fold challenge and sends it to the Prover
        fold_challenge = Fr.random_element()
        
        # The Prover computes and sends the f_fold_commit to the Verifier
        f_fold = split_and_fold(f, fold_challenge)
        f_fold_eval = [f_fold(w^(2*i)) for i in range(N//2)]
        f_fold_commit = Merkle.commit( f_fold_eval )

        # We store results
        f_commits += [f_commit]
        f_fold_commits += [f_fold_commit]
        f_evals += [f_eval]
        f_fold_evals += [f_fold_eval]
        fold_challenges += [fold_challenge]
            
        # Iterate
        N = N//2
        w = w^2
        f = f_fold
        
    return f_commits, f_fold_commits, f_evals, f_fold_evals, fold_challenges


# Returns the corresponding evaluations and Merkle proof for a certain query at index
def query(index, N, rounds, f_evals, f_fold_evals):

    f_l_values = []
    f_r_values = []
    f_fold_values = []
    f_l_value_proofs = []
    f_r_value_proofs = []
    f_fold_value_proofs = []
    
    for round in range(rounds):
                
        # The Prover retrieves the relevant evaluations
        f_l_value = f_evals[round][index]
        f_r_value = f_evals[round][N//2 + index]
        f_fold_value = f_fold_evals[round][index]

        # The Prover computes Merkle proofs for the evaluations
        f_l_value_proof = Merkle.open(index, f_evals[round])
        f_r_value_proof = Merkle.open(N//2 + index, f_evals[round])
        f_fold_value_proof = Merkle.open(index, f_fold_evals[round])

        # We store results
        f_l_values += [f_l_value]
        f_r_values += [f_r_value]
        f_fold_values += [f_fold_value]
        f_l_value_proofs += [f_l_value_proof]
        f_r_value_proofs += [f_r_value_proof]
        f_fold_value_proofs += [f_fold_value_proof]

        # Iterate
        N = N//2
        index = index % N//2

    return f_l_values, f_r_values, f_fold_values, f_l_value_proofs, f_r_value_proofs, f_fold_value_proofs

# Checks if a query at a certain index is consistent with the provided commitments
def verify(index, w, N, rounds, f_commits, f_fold_commits, fold_challenges, f_l_values, f_r_values, f_fold_values, f_l_value_proofs, f_r_value_proofs, f_fold_value_proofs, f_fold_eval_final):

    for round in range(rounds):

        # The Verifier checks the Merkle Tree's proofs
        assert Merkle.verify(f_commits[round], index, f_l_value_proofs[round], f_l_values[round])
        assert Merkle.verify(f_commits[round], N//2 + index, f_r_value_proofs[round], f_r_values[round])
        assert Merkle.verify(f_fold_commits[round], index, f_fold_value_proofs[round], f_fold_values[round])
        
        # The Verifier tests collinearity of the following points
        A = (w^index, f_l_values[round])
        B = (w^(N//2 + index), f_r_values[round])
        C = (fold_challenges[round], f_fold_values[round])
        assert are_collinear(A,B,C)

        # If it is the last rounds, the Verifier checks if the f_fold value matches the evaluation value returned by the Prover at the end of commit
        if round == rounds-1:
            assert f_fold_eval_final[index] == f_fold_values[round]

        # Iterate
        N = N//2
        w = w^2
        index = index % N//2

In [4]:
from math import log2

# The degree bound n = 2^k
k = 8
n = 2^k

# The RS codeword length
N = 2^4*n

# The code rate
rho = n/N

# The bits security level
𝛌 = 128

# The conjectured number of queries to reach the security level 𝛌
queries = ceil(𝛌/log2(rho^-1))

# The degree of the output folded polynomial after rounds Split & Fold 
rounds = k
final_degree = 2^(k-rounds) - 1

# We generates an N-th root of unity
w = setup(Fr, N)

# We generate a random polynomial
f = rand_poly(n)

# The Prover and Verifier engage in an interactive protocol to commit to the polynomial f
# The Verifier knows f_commits, f_fold_commits and fold_challenges, while all outputs are known to the Prover
f_commits, f_fold_commits, f_evals, f_fold_evals, fold_challenges = commit_ip(f, w, N, rounds)

# The Prover sends to the Verifier the last evaluation vector in full
f_fold_eval_final = f_fold_evals[-1]

# The Verifier checks if the last evaluation vector corresponds to a polynomial of degree equal final_degree
assert deg(f_fold_eval_final, w, N) <= final_degree

# The Verifier challenges the Prover by querying f at different indexes
for _ in range(queries):
    
    # The Verifier picks a random index and sends it to the Prover
    index = randint(0,N//2-1)
    
    # The Prover queries f and its folded polynomials at the relevant indexes and sends the values to the Verifier
    f_l_values, f_r_values, f_fold_values, f_l_value_proofs, f_r_value_proofs, f_fold_value_proofs = query(index,  N, rounds, f_evals, f_fold_evals)
    
    # The Verifier checks that the provided evaluations are consistent with the commitment to f
    verify(index, w, N, rounds, f_commits, f_fold_commits, fold_challenges, f_l_values, f_r_values, f_fold_values, f_l_value_proofs, f_r_value_proofs, f_fold_value_proofs, f_fold_eval_final)